In [1]:
import csv

In [2]:
def fix_position(arr):
    # x pos: av: -3, right: -6, left: -9
    # y pos: av: -2, right: -5, left:-8
    # pupil: av: -1, right: -4, left:-7
    #timestamp: -10
    
    arr[-10] = int(arr[-10])
    
    for i in range(3):
        if (arr[-9+i] == '.' or float(arr[-9+i]) == 0) and (arr[-6+i] == '.' or float(arr[-6+i]) == 0):
            arr[-9+i] = 0.0
            arr[-6+i] = 0.0
            arr[-3+i] = 0.0
        elif (arr[-9+i] == '.' or float(arr[-9+i]) == 0):
            val = float(arr[-6+i])
            arr[-9+i] = val
            arr[-6+i] = val
            arr[-3+i] = val
        elif (arr[-6+i] == '.' or float(arr[-6+i]) == 0):
            val = float(arr[-9+i])
            arr[-9+i] = val
            arr[-6+i] = val
            arr[-3+i] = val
        else:
            arr[-9+i] = float(arr[-9+i])
            arr[-6+i] = float(arr[-6+i])
            arr[-3+i] = 0.5*(arr[-6+i] + arr[-9+i])
    
    return arr
        
    

In [3]:
def is_junk(arr):
    junk = (len(arr) != 8 and len(arr) != 5) or (not arr[0].isdigit()) or any('.' not in arr[i] for i in range(1, len(arr)))
    if junk:
        return True
    
    if len(arr) == 5:
        temp = arr[4]
        arr[4] = arr[1]
        arr.append(arr[2])
        arr.append(arr[3])
        arr.append(temp)
    return False

In [4]:
def get_csv(file_path, participant):
    
    combined_header = ['Participant', 'Stimuli', 'Timestamp', 'Pos_X_Left', 'Pos_Y_Left', 'Pupil_Left',
                        'Pos_X_Right', 'Pos_Y_Right', 'Pupil_Right', 'Average_X', 'Average_Y', 'Average_Pupil']
    
    trial_headers = ['Participant', 'Question', 'Order', 'Trial', 'Timestamp', 'Pos_X_Left', 'Pos_Y_Left', 'Pupil_Left',
                        'Pos_X_Right', 'Pos_Y_Right', 'Pupil_Right', 'Average_X', 'Average_Y', 'Average_Pupil']
    
    combined_data = []
    
    current_trial_data = []
    current_question = 0
    current_trial = 0
    
    order = 0
    state = 0
    
    p = 'C:/Users/18475/Dropbox (UFL)/Infographic_study/Measurements/Main Learning Task/CSV/'
    #p = 'test/'
    
    with open(file_path, 'r') as fp:
        line = fp.readline()
        
        while line:
            tokens = line.split()
                      
            if state == 0: #initial useless stuff
                if tokens == ['SAMPLES', 'GAZE', 'LEFT', 'RIGHT', 'RATE', '1000.00', 'TRACKING', 'CR', 'FILTER', '1'] or tokens == ['SAMPLES', 'GAZE', 'RIGHT', 'RATE', '1000.00', 'TRACKING', 'CR', 'FILTER', '1'] or tokens == ['SAMPLES', 'GAZE', 'LEFT', 'RATE', '1000.00', 'TRACKING', 'CR', 'FILTER', '1']:
                        
                    fp.readline() #read the extra line
                    state  = 1
            
            elif state == 1: #junk before dot onset
                if len(tokens) == 5 and tokens[0] == 'MSG' and tokens[2] == 'Dot' and tokens[4] == 'Onset':
                    state = 2
                    order += 1
                    # add some file stuff
                
                elif is_junk(tokens):
                    pass
                
                else:
                    
                    combined_data.append(fix_position([participant, 'X' + str(order) + '0', tokens[0], tokens[1], tokens[2],
                                         tokens[3], tokens[4], tokens[5], tokens[6], 1, 1, 1]))
                
            elif state == 2: #dot onset just encountered
                if len(tokens) == 5 and tokens[0] == 'MSG' and tokens[2] == 'Dot' and tokens[4] == 'Offset':
                    state = 3
                
                elif is_junk(tokens):
                    pass
                
                else:
                    combined_data.append(fix_position([participant, 'DOT' + str(order), tokens[0], tokens[1], tokens[2],
                                         tokens[3], tokens[4], tokens[5], tokens[6], 1, 1, 1]))
                
            elif state == 3: #junk adter dot offset
                if len(tokens) == 5 and tokens[0] == 'MSG' and tokens[2] == 'Trial':
                    state = 4
                    current_question = int(tokens[3])
                    current_trial = int(tokens[4][-1])
                    current_trial_data = []
                    
                elif is_junk(tokens):
                    pass
                
                else:
                    #add to a file
                    combined_data.append(fix_position([participant, 'X' + str(order%12) + '1', tokens[0], tokens[1], tokens[2],
                                         tokens[3], tokens[4], tokens[5], tokens[6], 1, 1, 1]))
                
            elif state == 4: #a step of a trial
                if len(tokens) == 5 and tokens[0] == 'MSG' and tokens[2] == 'Trial':
                    #same state
                    #dump to a csv
                    path = p + 'P_' + str(participant) + '_Qorder_' + str(order) + '_T_' + str(current_trial) + '.csv'
                    with open(path, 'w', encoding='UTF8', newline='') as f:
                        writer = csv.writer(f)

                        # write the header
                        writer.writerow(trial_headers)

                        # write multiple rows
                        writer.writerows(current_trial_data)
                    #restore
                    current_question = int(tokens[3])
                    current_trial = int(tokens[4][-1])
                    current_trial_data = []
                    
                
                elif len(tokens) == 5 and tokens[0] == 'MSG' and tokens[2] == 'Question' and tokens[4] == 'Offset':
                    #end of the question
                    #dump to a csv
                    path = p + 'P_' + str(participant) + '_Qorder_' + str(order) + '_T_' + str(current_trial) + '.csv'
                    with open(path, 'w', encoding='UTF8', newline='') as f:
                        writer = csv.writer(f)

                        # write the header
                        writer.writerow(trial_headers)

                        # write multiple rows
                        writer.writerows(current_trial_data)
                    #retore
                    current_trial_data = []
                    state = 1
                    
                elif is_junk(tokens):
                    pass
                
                else:
                    #dump to the csv
                    combined_data.append(fix_position([participant, 'Q' + str(current_question) + 'T' + str(current_trial), tokens[0], tokens[1], tokens[2],
                                         tokens[3], tokens[4], tokens[5], tokens[6], 1, 1, 1]))
                    current_trial_data.append(fix_position([participant, current_question, order, current_trial, tokens[0], tokens[1], tokens[2],
                                         tokens[3], tokens[4], tokens[5], tokens[6], 1, 1, 1]))
                    
                
            
            
            line = fp.readline()
    
    path = p + 'P_' + str(participant) + '_Combined.csv'
    with open(path, 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)

        # write the header
        writer.writerow(combined_header)

        # write multiple rows
        writer.writerows(combined_data)
        
        

In [5]:
#get_csv(/sub_51.asc', 51)
base = 'C:/Users/18475/Dropbox (UFL)/Infographic_study/Measurements/Main Learning Task/ASC/'
for i in range(1, 52):
    f = 'sub_'
    if i < 10:
        f = base + f + '0' + str(i) + '.asc'
    else:
        f = base + f  + str(i) + '.asc'
        
    get_csv(f, i)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/18475/Dropbox (UFL)/Infographic_study/Measurements/Main Learning Task/ASC/sub_01.asc'